# Friday 3.1
The ideas of Friday 3, but maybe it will work

## Abstract:
Friday is a digital system designed to be a starting point for a truewly useful, JARVIS level, assistant. This will combine the lastest of Genreative learning, input proccessing, and self supervised learning.

The hope of friday is that a minimal model can branch out using the internet to master specific skills and while activly learning from mistakes and successes.

This will all be done on a single, interconnected model written in torch.

Let's get going....

## Imports
First thing we need to do is import all the neccesary modules. We want to use primarly large, established projects, but if we need code from a small one we'll use it.

In [2]:
import sys
from os.path import exists
import wikipediaapi
import torch
import torch.nn as nn
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer,  AutoModelForSeq2SeqLM, AutoModelWithLMHead

## Constants
Just some variables that need to be defined and not changed

In [2]:
# Query types
QUERY_TYPE_GENERAL = 0
QUERY_TYPE_DEFINITION = 1
QUERY_TYPE_DETAILS = 2
QUERY_TYPE_EXAMPLES = 3

# Data retrieval types
# Will be used for retrieving specific data by title
DATA_TYPE_BOOK = 0
DATA_TYPE_ARTICLE = 1
DATA_TYPE_WEBPAGE = 2
DATA_TYPE_CODE = 3

## Paramters
Tunable paramters that allow you to change the way friday operates

In [3]:
NAME = "Friday"
PROMPT = "Hello, I'm Friday. What can I do for you?"
FRIDAY_LIBRARY = "./friday.cvs" # A refrence database built solely by the model for data storage - fridays notebook

## Utilty's
The boring stuff, logging and other things

In [4]:
def log(area, message):
    print("%s:" % area, message)


# The Internet Brain
The tools that will link a brain to the internet

The data format the ask() function will return

My little Wikipedia utilty I use sometimes

In [5]:
class WikiBrain:
    def __init__(self, lang):
        self.wiki = wikipediaapi.Wikipedia(lang)

    def pageexist(self, term):
        return self.wiki.page(term)
    
    def summary(self, page):
        return self.wiki.page(page).summary
    def full_page(self, page):
        return self.wiki.page(page).text

In [6]:
wb = WikiBrain("en")

# Ask function
Super basic now, but will expand it as needed

In [7]:
def ask(query, second_pass:bool = False):
    if(second_pass): #If more detailed data is needed - most likly if the first pass of this function didn't work
        return wb.full_page(query)
    return wb.summary(query)


## Action function
This will be used to let Friday interact with the outside world

## Fact storage function

In [ ]:
def get_fl():
    if(exists(FRIDAY_LIBRARY)):
        return pd.read_csv(FRIDAY_LIBRARY)
    # Create the dataframe
    return pd.DataFrame({"fact": pd.Series(), "context": pd.Series}) # Not the final form 

# Friday layers
The extra layer that makes friday friday

In [ ]:
class friday_layer (nn.Module):

    def __init__(self, model): # Could be gpt or t0pp idk yet
        self.model = model
        # Do stuff lol
    # kinda ripping https://github.com/Himabindugssn/Sentiment-classification-using-transformers/blob/main/BERTFinetuning.ipynb
    def forward(self, sent_id, mask):
        _, cls_hs = self.model(sent_id, attention_mask=mask, return_dict=False)

## Data model

Load up the model and fine tune it

In [1]:
#tokenizer = AutoTokenizer.from_pretrained("bigscience/T0_3B")
#model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0_3B")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large") # use this for now, will probably fine tune GPT-j or t0pp
model =AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-large")


NameError: name 'AutoTokenizer' is not defined

We'll fine tune on my thursday dataset.... 

## Algorithm

For now it's just a simple chat bot - we'll need to add on the fly learning at some point (using ask() to get the data)

In [ ]:
step = 0
while True:
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    convinput = input()
    if(convinput == "stop"):
        break
    new_user_input_ids = tokenizer.encode(convinput + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response 
    chat_history_ids = model.generate(
        bot_input_ids,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=100,
        top_p=0.7,
        temperature=0.8
    )
    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))